# GP Prior Parameters

This notebook explores how different GP kernel parameters affect prior draws
on triangular fault meshes. It is self-contained and does **not** require
loading any MCMC run.

We demonstrate two activation regimes:

1. **Linear / unconstrained** (mesh 3 — BEL elastic fault)
2. **Sigmoid / constrained** (mesh 0 — Cascadia coupling, bounded to [0, 1])

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from celeri.mesh import Mesh, MeshConfig, _get_eigenvalues_and_eigenvectors
from celeri.plot import plot_mesh
from celeri.solve_mcmc import _constrain_field, _unconstrain_field

## Load meshes

In [ ]:
# Mesh 3: BEL elastic fault (376 TDE, unconstrained)
mesh_3 = Mesh.from_params(
    MeshConfig(
        file_name=Path("doc_gp_params.ipynb"),
        mesh_filename=Path("../wna/data/mesh/bel_b50_d20_s100.msh"),
        n_modes_strike_slip=30,
        n_modes_dip_slip=30,
        matern_nu=2.5,  # default: mcmc_default_mesh_matern_nu
        matern_length_scale=0.2,
        matern_length_units="diameters",
        eigenvector_algorithm="eigh",
    )
)
print(f"Mesh 3: {mesh_3.n_tde} TDE")

In [ ]:
# Mesh 0: Cascadia coupling (1841 TDE, constrained [0, 1])
mesh_0 = Mesh.from_params(
    MeshConfig(
        file_name=Path("doc_gp_params.ipynb"),
        mesh_filename=Path("../wna/data/mesh/cascadia.msh"),
        n_modes_strike_slip=10,
        n_modes_dip_slip=50,
        matern_nu=2.5,  # default: mcmc_default_mesh_matern_nu
        matern_length_scale=0.2,
        matern_length_units="diameters",
        eigenvector_algorithm="eigh",
    )
)
print(f"Mesh 0: {mesh_0.n_tde} TDE")

## Helper functions

In [ ]:
def draw_prior(
    eigenvectors: np.ndarray,
    eigenvalues: np.ndarray,
    n_eigs: int,
    sigma: float,
    rng: np.random.Generator,
    n_draws: int = 10,
    *,
    uniform_weights: bool = False,
) -> np.ndarray:
    """Generate prior draws from a GP eigenmode expansion.

    Returns an (n_tde, n_draws) array of field values.
    If uniform_weights=True, all eigenmodes are weighted equally (white noise prior).
    """
    coefs = rng.normal(size=(n_eigs, n_draws))
    if not uniform_weights:
        coefs *= sigma * np.sqrt(eigenvalues[:n_eigs, None])
    else:
        coefs *= sigma
    return eigenvectors[:, :n_eigs] @ coefs


def constrain(values: np.ndarray, lower: float, upper: float) -> np.ndarray:
    """Apply sigmoid constraint via _constrain_field, returning numpy."""
    return _constrain_field(values, lower, upper).eval()


def plot_draws_grid(
    mesh: Mesh,
    draws: np.ndarray,
    title: str,
    *,
    subtitles: list[str] | None = None,
    cmap: str = "seismic",
    center: float | None = 0,
    vmin: float | None = None,
    vmax: float | None = None,
    ncols: int = 5,
) -> None:
    """Plot a grid of field draws on a mesh."""
    n = draws.shape[1]
    nrows = int(np.ceil(n / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(2.4 * ncols, 4 * nrows))
    axes_flat = np.atleast_1d(axes).flat

    for i, ax in enumerate(axes_flat):
        if i >= n:
            ax.set_visible(False)
            continue
        pc = plot_mesh(
            mesh,
            fill_value=draws[:, i],
            ax=ax,
            cmap=cmap,
            center=center,
            vmin=vmin,
            vmax=vmax,
            set_limits=True,
        )
        if subtitles is not None:
            ax.set_title(subtitles[i], fontsize=10)

    cbar = fig.colorbar(
        pc,
        ax=list(axes_flat),
        orientation="vertical",
        fraction=0.025,
        pad=0.04,
    )
    cbar.set_label("value", rotation=270, labelpad=20)
    fig.suptitle(title, fontsize=16)
    plt.tight_layout()

---

## 1. Linear / Unconstrained Prior Draws (Mesh 3 — Elastic)

### 1a. Eigenmodes

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(12, 8))
vmin, vmax = -0.1, 0.1

for ax, mode in zip(axes.flat, range(10)):
    plot_mesh(
        mesh_3,
        fill_value=mesh_3.eigenvectors[:, mode],
        ax=ax,
        vmin=vmin,
        vmax=vmax,
        set_limits=True,
    )
    ax.set_title(f"mode {mode}", fontsize=10)

fig.suptitle(r"First 10 Matérn $\nu=5/2$ eigenmodes (mesh 3)", fontsize=16)
plt.tight_layout()

### 1b. Effect of number of eigenmodes

As the number of eigenmodes increases, Matérn priors retain spatial
smoothness because higher-order modes are down-weighted by their
eigenvalues.  A uniformly-weighted (white noise) prior converges to
spatially uncorrelated noise.

In [ ]:
# Compute eigendecomposition with the maximum number of modes
n_max = mesh_3.n_tde - 1  # 375 for 376 TDE

eig_cache_3: dict[float, tuple[np.ndarray, np.ndarray]] = {}
for nu in [0.5, 1.5, 2.5]:
    eig_cache_3[nu] = _get_eigenvalues_and_eigenvectors(
        n_eigenvalues=n_max,
        x=mesh_3.x_centroid,
        y=mesh_3.y_centroid,
        z=mesh_3.z_centroid,
        matern_nu=nu,
        matern_length_scale=0.2,
        matern_length_units="diameters",
    )
    print(f"nu={nu}: computed {n_max} eigenmodes")

In [ ]:
n_eigs_list = [1, 5, 10, 25, 50, 100, 150, 200, 250, 350]
sigma = 5.0  # default: mcmc_default_mesh_elastic_sigma

kernel_labels = [
    ("White Noise (uniform weights)", None, True),
    (r"Matérn $\nu=1/2$", 0.5, False),
    (r"Matérn $\nu=3/2$", 1.5, False),
    (r"Matérn $\nu=5/2$", 2.5, False),
]

for label, nu, uniform in kernel_labels:
    rng = np.random.default_rng(42)

    if uniform:
        # White noise: use eigenvectors from nu=0.5 (arbitrary, only directions matter)
        eigenvalues, eigenvectors = eig_cache_3[0.5]
    else:
        eigenvalues, eigenvectors = eig_cache_3[nu]

    # Draw one realisation with the maximum number of modes
    coefs = rng.normal(size=n_max)
    if not uniform:
        coefs *= sigma * np.sqrt(eigenvalues[:n_max])
    else:
        coefs *= sigma

    fig, axes = plt.subplots(2, 5, figsize=(12, 8))
    for ax, n_eigs in zip(axes.flat, n_eigs_list):
        field = eigenvectors[:, :n_eigs] @ coefs[:n_eigs]
        plot_mesh(mesh_3, fill_value=field, ax=ax, set_limits=True)
        ax.set_title(f"n_eigs = {n_eigs}", fontsize=10)

    fig.suptitle(f"{label} — increasing eigenmodes (mesh 3)", fontsize=14)
    plt.tight_layout()

### 1c. Effect of smoothness (nu)

Higher $\nu$ produces smoother fields. $\nu=1/2$ is exponential
(rough, continuous but not differentiable), $\nu=3/2$ is
once-differentiable, $\nu=5/2$ is twice-differentiable.

In [ ]:
n_eigs = 30
sigma = 5.0  # default: mcmc_default_mesh_elastic_sigma
n_draws = 5
n_kernels = len(kernel_labels)

fig, axes = plt.subplots(n_draws, n_kernels, figsize=(2.4 * n_kernels, 4 * n_draws))

for col, (label, nu, uniform) in enumerate(kernel_labels):
    rng = np.random.default_rng(42)
    if uniform:
        eigenvalues, eigenvectors = eig_cache_3[0.5]
    else:
        eigenvalues, eigenvectors = eig_cache_3[nu]

    draws = draw_prior(eigenvectors, eigenvalues, n_eigs, sigma, rng, n_draws, uniform_weights=uniform)

    for row in range(n_draws):
        ax = axes[row, col]
        plot_mesh(mesh_3, fill_value=draws[:, row], ax=ax, set_limits=True)
        if row == 0:
            ax.set_title(label, fontsize=10)
        if col == 0:
            ax.set_ylabel(f"draw {row}", fontsize=12)

fig.suptitle(f"Effect of smoothness (n_eigs={n_eigs}, sigma={sigma})", fontsize=16)
plt.tight_layout()

### 1d. Effect of length scale

The length scale controls the spatial correlation distance. Larger
length scales produce broader, smoother features. Length scale is
specified in units of mesh diameters.

In [ ]:
length_scale_values = [0.05, 0.1, 0.25, 0.5, 1.0]
n_eigs = 30
sigma = 5.0  # default: mcmc_default_mesh_elastic_sigma
n_draws = 5
n_ls = len(length_scale_values)

# Pre-compute eigenmodes for each length scale
eigs_by_ls = {}
for ls in length_scale_values:
    eigs_by_ls[ls] = _get_eigenvalues_and_eigenvectors(
        n_eigenvalues=n_eigs,
        x=mesh_3.x_centroid,
        y=mesh_3.y_centroid,
        z=mesh_3.z_centroid,
        matern_nu=2.5,
        matern_length_scale=ls,
        matern_length_units="diameters",
    )

fig, axes = plt.subplots(n_draws, n_ls, figsize=(2.4 * n_ls, 4 * n_draws))

for col, ls in enumerate(length_scale_values):
    eigenvalues_ls, eigenvectors_ls = eigs_by_ls[ls]
    rng = np.random.default_rng(42)
    draws = draw_prior(eigenvectors_ls, eigenvalues_ls, n_eigs, sigma, rng, n_draws)

    for row in range(n_draws):
        ax = axes[row, col]
        plot_mesh(mesh_3, fill_value=draws[:, row], ax=ax, set_limits=True)
        if row == 0:
            ax.set_title(f"$\\ell={ls}$", fontsize=10)
        if col == 0:
            ax.set_ylabel(f"draw {row}", fontsize=12)

fig.suptitle(r"Effect of length scale $\ell$ (Matérn $\nu=5/2$, mesh 3)", fontsize=16)
plt.tight_layout()

---

## 2. Sigmoid / Constrained Prior Draws (Mesh 0 — Coupling)

In [ ]:
# Default coupling prior mean: 0.9 in constrained [0, 1] space
# Map to unconstrained space for the GP parameterization
coupling_mean_constrained = 0.9  # default: mcmc_default_mesh_coupling_mean
mu_unconstrained = float(_unconstrain_field(
    np.array([coupling_mean_constrained]), lower=0.0, upper=1.0
)[0])
print(f"Coupling mean: {coupling_mean_constrained} (constrained) → {mu_unconstrained:.4f} (unconstrained)")

### 2a. Eigenmodes

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(12, 8))
vmin, vmax = -0.1, 0.1

for ax, mode in zip(axes.flat, range(10)):
    plot_mesh(
        mesh_0,
        fill_value=mesh_0.eigenvectors[:, mode],
        ax=ax,
        vmin=vmin,
        vmax=vmax,
        set_limits=True,
    )
    ax.set_title(f"mode {mode}", fontsize=10)

fig.suptitle(r"First 10 Matérn $\nu=5/2$ eigenmodes (mesh 0)", fontsize=16)
plt.tight_layout()

### 2b. Effect of number of eigenmodes with sigmoid

The sigmoid activation squashes the unconstrained GP field into
$[0, 1]$. With few eigenmodes, the field has broad spatial structure.
With many, Matérn priors stay smooth while white noise becomes
spatially uncorrelated.

In [ ]:
# Compute eigendecomposition for mesh 0 with many modes
n_max_0 = min(mesh_0.n_tde - 1, 1500)

eig_cache_0: dict[float, tuple[np.ndarray, np.ndarray]] = {}
for nu in [0.5, 1.5, 2.5]:
    eig_cache_0[nu] = _get_eigenvalues_and_eigenvectors(
        n_eigenvalues=n_max_0,
        x=mesh_0.x_centroid,
        y=mesh_0.y_centroid,
        z=mesh_0.z_centroid,
        matern_nu=nu,
        matern_length_scale=0.2,
        matern_length_units="diameters",
    )
    print(f"nu={nu}: computed {n_max_0} eigenmodes")

In [ ]:
n_eigs_list_0 = [1, 5, 10, 25, 50, 100, 250, 500, 1000, 1500]
sigma = 1.0  # default: mcmc_default_mesh_coupling_sigma

for label, nu, uniform in kernel_labels:
    rng = np.random.default_rng(42)

    if uniform:
        eigenvalues, eigenvectors = eig_cache_0[0.5]
    else:
        eigenvalues, eigenvectors = eig_cache_0[nu]

    coefs = rng.normal(size=n_max_0)
    if not uniform:
        coefs *= sigma * np.sqrt(eigenvalues[:n_max_0])
    else:
        coefs *= sigma

    fig, axes = plt.subplots(2, 5, figsize=(12, 8))
    for ax, n_eigs in zip(axes.flat, n_eigs_list_0):
        field = eigenvectors[:, :n_eigs] @ coefs[:n_eigs]
        field_constrained = constrain(field + mu_unconstrained, 0.0, 1.0)
        plot_mesh(mesh_0, fill_value=field_constrained, ax=ax, vmin=0, vmax=1, center=0.5, set_limits=True)
        ax.set_title(f"n_eigs = {n_eigs}", fontsize=10)

    fig.suptitle(f"{label} — sigmoid [0, 1] — increasing eigenmodes (mesh 0)", fontsize=14)
    plt.tight_layout()

### 2c. Effect of smoothness (nu) with sigmoid

In [ ]:
n_eigs = 50
sigma = 1.0  # default: mcmc_default_mesh_coupling_sigma
n_draws = 5
n_kernels = len(kernel_labels)

fig, axes = plt.subplots(n_draws, n_kernels, figsize=(2.4 * n_kernels, 4 * n_draws))

for col, (label, nu, uniform) in enumerate(kernel_labels):
    rng = np.random.default_rng(42)
    if uniform:
        eigenvalues, eigenvectors = eig_cache_0[0.5]
    else:
        eigenvalues, eigenvectors = eig_cache_0[nu]

    draws = draw_prior(eigenvectors, eigenvalues, n_eigs, sigma, rng, n_draws, uniform_weights=uniform)
    draws_constrained = constrain(draws + mu_unconstrained, 0.0, 1.0)

    for row in range(n_draws):
        ax = axes[row, col]
        plot_mesh(mesh_0, fill_value=draws_constrained[:, row], ax=ax, vmin=0, vmax=1, center=0.5, set_limits=True)
        if row == 0:
            ax.set_title(label, fontsize=10)
        if col == 0:
            ax.set_ylabel(f"draw {row}", fontsize=12)

fig.suptitle(f"Effect of smoothness — sigmoid [0, 1] (n_eigs={n_eigs}, sigma={sigma})", fontsize=16)
plt.tight_layout()

### 2d. Effect of amplitude (sigma) with sigmoid

Larger $\sigma$ pushes the unconstrained field further from the
midpoint, causing the sigmoid to saturate toward 0 or 1.

In [ ]:
n_eigs = 50
sigma_values = [0.1, 0.25, 0.5, 1.0, 2.0]
n_draws = 5
n_sigma = len(sigma_values)
eigenvalues, eigenvectors = eig_cache_0[2.5]  # default: mcmc_default_mesh_matern_nu

fig, axes = plt.subplots(n_draws, n_sigma, figsize=(2.4 * n_sigma, 4 * n_draws))

for col, sigma in enumerate(sigma_values):
    rng = np.random.default_rng(42)
    draws = draw_prior(eigenvectors, eigenvalues, n_eigs, sigma, rng, n_draws)
    draws_constrained = constrain(draws + mu_unconstrained, 0.0, 1.0)

    for row in range(n_draws):
        ax = axes[row, col]
        plot_mesh(mesh_0, fill_value=draws_constrained[:, row], ax=ax, vmin=0, vmax=1, center=0.5, set_limits=True)
        if row == 0:
            ax.set_title(f"$\\sigma={sigma}$", fontsize=10)
        if col == 0:
            ax.set_ylabel(f"draw {row}", fontsize=12)

fig.suptitle(r"Effect of amplitude $\sigma$ — sigmoid [0, 1] (Matérn $\nu=5/2$, mesh 0)", fontsize=16)
plt.tight_layout()

### 2e. Effect of length scale with sigmoid

In [ ]:
length_scale_values = [0.05, 0.1, 0.25, 0.5, 1.0]
n_eigs = 50
sigma = 1.0  # default: mcmc_default_mesh_coupling_sigma
n_draws = 5
n_ls = len(length_scale_values)

# Pre-compute eigenmodes for each length scale
eigs_by_ls_0 = {}
for ls in length_scale_values:
    eigs_by_ls_0[ls] = _get_eigenvalues_and_eigenvectors(
        n_eigenvalues=n_eigs,
        x=mesh_0.x_centroid,
        y=mesh_0.y_centroid,
        z=mesh_0.z_centroid,
        matern_nu=2.5,
        matern_length_scale=ls,
        matern_length_units="diameters",
    )

fig, axes = plt.subplots(n_draws, n_ls, figsize=(2.4 * n_ls, 4 * n_draws))

for col, ls in enumerate(length_scale_values):
    eigenvalues_ls, eigenvectors_ls = eigs_by_ls_0[ls]
    rng = np.random.default_rng(42)
    draws = draw_prior(eigenvectors_ls, eigenvalues_ls, n_eigs, sigma, rng, n_draws)
    draws_constrained = constrain(draws + mu_unconstrained, 0.0, 1.0)

    for row in range(n_draws):
        ax = axes[row, col]
        plot_mesh(mesh_0, fill_value=draws_constrained[:, row], ax=ax, vmin=0, vmax=1, center=0.5, set_limits=True)
        if row == 0:
            ax.set_title(f"$\\ell={ls}$", fontsize=10)
        if col == 0:
            ax.set_ylabel(f"draw {row}", fontsize=12)

fig.suptitle(r"Effect of length scale $\ell$ — sigmoid [0, 1] (Matérn $\nu=5/2$, mesh 0)", fontsize=16)
plt.tight_layout()

### 2f. Effect of coupling mean

The prior mean shifts the unconstrained field before the sigmoid is
applied. A mean of 0 maps to 0.5 coupling; larger means bias the
prior toward full coupling (1.0).

In [ ]:
mean_values_constrained = [0.1, 0.3, 0.5, 0.7, 0.9]
n_eigs = 50
sigma = 1.0  # default: mcmc_default_mesh_coupling_sigma
n_draws = 5
n_means = len(mean_values_constrained)
eigenvalues, eigenvectors = eig_cache_0[2.5]  # default: mcmc_default_mesh_matern_nu

# Map each constrained mean to unconstrained space
mu_values = {
    m: float(_unconstrain_field(np.array([m]), lower=0.0, upper=1.0)[0])
    for m in mean_values_constrained
}

fig, axes = plt.subplots(n_draws, n_means, figsize=(2.4 * n_means, 4 * n_draws))

for col, mean_c in enumerate(mean_values_constrained):
    mu = mu_values[mean_c]
    rng = np.random.default_rng(42)
    draws = draw_prior(eigenvectors, eigenvalues, n_eigs, sigma, rng, n_draws)
    draws_constrained = constrain(draws + mu, 0.0, 1.0)

    for row in range(n_draws):
        ax = axes[row, col]
        plot_mesh(mesh_0, fill_value=draws_constrained[:, row], ax=ax, vmin=0, vmax=1, center=0.5, set_limits=True)
        if row == 0:
            ax.set_title(f"mean={mean_c}", fontsize=10)
        if col == 0:
            ax.set_ylabel(f"draw {row}", fontsize=12)

fig.suptitle(r"Effect of coupling mean — sigmoid [0, 1] (Matérn $\nu=5/2$, mesh 0)", fontsize=16)
plt.tight_layout()

### 2g. Unconstrained vs constrained comparison

Same draws shown before (left) and after (right) sigmoid transformation.

In [ ]:
n_eigs = 50
sigma = 1.0  # default: mcmc_default_mesh_coupling_sigma
n_draws = 5
eigenvalues, eigenvectors = eig_cache_0[2.5]  # default: mcmc_default_mesh_matern_nu

rng = np.random.default_rng(42)
draws = draw_prior(eigenvectors, eigenvalues, n_eigs, sigma, rng, n_draws)
draws_with_mean = draws + mu_unconstrained
draws_constrained = constrain(draws_with_mean, 0.0, 1.0)

fig, axes = plt.subplots(n_draws, 2, figsize=(2.4 * 2, 4 * n_draws))

for row in range(n_draws):
    # Left column: unconstrained (with mean offset)
    ax = axes[row, 0]
    plot_mesh(mesh_0, fill_value=draws_with_mean[:, row], ax=ax, set_limits=True)
    if row == 0:
        ax.set_title("Unconstrained", fontsize=10)
    ax.set_ylabel(f"draw {row}", fontsize=12)

    # Right column: constrained
    ax = axes[row, 1]
    plot_mesh(mesh_0, fill_value=draws_constrained[:, row], ax=ax, vmin=0, vmax=1, center=0.5, set_limits=True)
    if row == 0:
        ax.set_title("Sigmoid [0, 1]", fontsize=10)

fig.suptitle(r"Unconstrained vs sigmoid — Matérn $\nu=5/2$ (mesh 0, mean=0.9)", fontsize=16)
plt.tight_layout()